In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("distilgpt2")  # ~82M params
tok = GPT2Tokenizer.from_pretrained("distilgpt2")


In [8]:

total = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total/1e6, trainable/1e6) 

81.912576 81.912576


In [11]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset
import math

# Load (your pruned model OR distilgpt2 if debugging)
# model = your_pruned_model
# model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# tok = GPT2Tokenizer.from_pretrained("distilgpt2")
tok.pad_token = tok.eos_token  # important for padding

# Load dataset
ds = load_dataset("wikitext", "wikitext-2-raw-v1")

def preprocess(ex):
    return tok(ex["text"], truncation=True, padding="max_length", max_length=128)

tok_ds = ds.map(preprocess, batched=True, remove_columns=["text"])

# ✅ THIS is what fixes your error
collator = DataCollatorForLanguageModeling(
    tokenizer=tok,
    mlm=False  # because GPT2 is causal LM, not masked LM
)

# Training args
args = TrainingArguments(
    output_dir="./gpt2-run",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_ds["train"],
    eval_dataset=tok_ds["validation"],
    data_collator=collator,
)

trainer.train()

# Perplexity
res = trainer.evaluate()
print(f"Eval Loss: {res['eval_loss']:.4f}")
print(f"Perplexity: {math.exp(res['eval_loss']):.2f}")


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.462100,3.561412


Eval Loss: 3.5614
Perplexity: 35.21


In [12]:
import torch, numpy as np
from transformers import (
    GPT2Tokenizer, GPT2ForSequenceClassification,
    Trainer, TrainingArguments
)
from datasets import load_dataset
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------
# 1. Load IMDb Dataset
# --------------------------
ds = load_dataset("imdb")
ds = ds.shuffle(seed=42)

# --------------------------
# 2. Tokenizer (GPT-2 / DistilGPT2)
# --------------------------
tok = GPT2Tokenizer.from_pretrained("distilgpt2")
tok.pad_token = tok.eos_token

def tokenize(batch):
    return tok(
        batch["text"], truncation=True, padding="max_length",
        max_length=128
    )

ds_tok = ds.map(tokenize, batched=True)
ds_tok.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

train_ds = ds_tok["train"].select(range(20000))   # smaller subset for speed
test_ds  = ds_tok["test"].select(range(5000))

# --------------------------
# 3. Load Distilled GPT-2 for Classification
# --------------------------
def load_distil_model():
    model = GPT2ForSequenceClassification.from_pretrained(
        "distilgpt2",
        num_labels=2,
        pad_token_id=tok.eos_token_id
    )
    return model.to(device)

# --------------------------
# 4. Metrics
# --------------------------
acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return acc.compute(predictions=preds, references=labels)

# --------------------------
# 5. Train & Evaluate DistilGPT2
# --------------------------
print("\n🔹 Training DistilGPT2 ...")
model = load_distil_model()

args = TrainingArguments(
    output_dir="./results_distilgpt2",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=200
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics
)

trainer.train()
result = trainer.evaluate()
print(f"✅ DistilGPT2 Accuracy:", result["eval_accuracy"])


Map: 100%|██████████| 50000/50000 [00:50<00:00, 982.97 examples/s] 



🔹 Training DistilGPT2 ...


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.819400,0.660719,0.860200


✅ DistilGPT2 Accuracy: 0.8602


In [13]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import evaluate
import numpy as np
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------
# 1. Dataset: CNN/DailyMail (first 100 test samples)
# --------------------------
data = load_dataset("cnn_dailymail", "3.0.0", split="test[:100]")  

# --------------------------
# 2. ROUGE Metric
# --------------------------
rouge = evaluate.load("rouge")

# --------------------------
# 3. Tokenizer (same for all GPT-based models)
# --------------------------
tok = GPT2Tokenizer.from_pretrained("gpt2")
tok.pad_token = tok.eos_token

# --------------------------
# 4. Models to Evaluate
# --------------------------
models = {
    "distilgpt2": "distilgpt2"   # ✅ added distilled model
}

# --------------------------
# 5. Summarization Function
# --------------------------
def generate_summary(model, text, max_new_tokens=60):
    inp = tok(
        text, return_tensors="pt",
        truncation=True, padding=True, max_length=128
    ).to(device)

    output = model.generate(
        **inp,
        max_new_tokens=max_new_tokens,
        num_beams=3,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    return tok.decode(output[0], skip_special_tokens=True)

# --------------------------
# 6. Evaluation Loop
# --------------------------
results = {}

for name, path in models.items():
    print(f"\n🔹 Evaluating ROUGE for: {name}")
    
    model = GPT2LMHeadModel.from_pretrained(path).to(device).eval()
    predictions, references = [], []

    for i in tqdm(range(len(data))):
        article = data[i]["article"]
        ref = data[i]["highlights"]
        pred = generate_summary(model, article)
        predictions.append(pred)
        references.append(ref)

    score = rouge.compute(predictions=predictions, references=references)
    results[name] = score
    print(f"✅ ROUGE for {name}:", score)

# --------------------------
# 7. Final Comparison
# --------------------------
print("\n📊 Final ROUGE-L Comparison:")
for model_name, score in results.items():
    print(model_name, "→ ROUGE-L:", score["rougeL"])


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: cfcbe770-7794-45f5-8e56-5a821a608e80)')' thrown while requesting HEAD https://huggingface.co/datasets/cnn_dailymail/resolve/main/README.md
Retrying in 1s [Retry 1/5].
Generating test split: 100%|██████████| 11490/11490 [00:00<00:00, 91882.50 examples/s]



🔹 Evaluating ROUGE for: distilgpt2


100%|██████████| 100/100 [00:20<00:00,  4.86it/s]


✅ ROUGE for distilgpt2: {'rouge1': np.float64(0.22278047339844204), 'rouge2': np.float64(0.09867662047952458), 'rougeL': np.float64(0.16022128844320674), 'rougeLsum': np.float64(0.188834834282039)}

📊 Final ROUGE-L Comparison:
distilgpt2 → ROUGE-L: 0.16022128844320674
